# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования:** 
- Проверка гипотезы: на надежность клиента банка влияет наличие у него детей.
- Проверка гипотезы: на надежность клиента банка влияет семеное положение

## Шаг 1. Откройте файл с данными и изучите общую информацию

## Предварительное изучение

In [80]:
import pandas as pd
import re
import pymorphy2
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [81]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [82]:
df.columns  #Проверка на кривизну названий столбцов

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [83]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [84]:
for i in ('income_type', 'children', 'family_status', 'family_status_id', 'education', 'education_id', 'debt'):
   print(f'\n\n-------------Информация {i}------------\n ')
   display(df[i].value_counts()) 



-------------Информация income_type------------
 


сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64



-------------Информация children------------
 


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64



-------------Информация family_status------------
 


...

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64



-------------Информация education_id------------
 


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64



-------------Информация debt------------
 


0    19784
1     1741
Name: debt, dtype: int64

In [85]:
df.sort_values(by=['total_income'], ascending=False) #чтоб не копировать код, тут же проверял и минимальные

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [86]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**  
- `days_employed` Не очень понятно что с графой трудовой стаж и откуда растут ноги... по идеи значение должно быть int в днях... а сейчас какая то билиберда(ушл к преподу). Ну еще пропуски есть... 
- `income_type` Не понятен статус "сотрудник", если сотрудние банка: то надо его к черту вычеркнуть... 
- `children` 20детей... надо будет глянуть на рекордсмена и либо удалить либо оставить если это женщина из Воронежа. -1 тоже так себе показатель... может беременных занесли или еще чего.
- `education` Образование надо чистить - разные регисторы, может и орфографические ошибки.
- `education_id` Надо сравнить с education ... Если все соответсвует, то просто столбец со str значением удалить.
- `dob_years` По хорошему надо сгруппировать не несколько статусов.Также есть новорожденные по возрасту... надо посмотреть сколько их, если мало удалить, если много - то куда то распихать
- `total_income` есть пропуск зарплат... не очень понятно пока что с этим делать. и формат еще экспоненциальный "Эх... Чтоб у меня зарпалата также записывалась( 
- `family_status` и `family_status_id` внешне все хорошо при том что одни из самых важных ячеек в нашем исследовании
- `debt` Вроде все хорошо.
- `purpose` цель кредита: не уверен что вообще столбец нужен для поставленной задачи исследований. Разве что факультативно.

удивительно но столбцы названы корректно...  

## Шаг 2. Предобработка данных

### Обработка пропусков

In [87]:
df[df['total_income'] < 0]  # не затисались нули в зарплаты... будем верить полученным данным

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [88]:
df[df['total_income'].isna()].head(5) # просто посмотрел у кого пропуски... пропуски случайные(или не вижу закономерности, делал сначала head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [89]:

df_mediana_income = df.groupby('income_type')['total_income'].median()  # Делаем df с медианными значениями

In [90]:
def change_income(row):
    '''Check income, if NoneType in ['total_income'] return mediana from dataFrame with mediana value, else return total_income
    Args:
        row (row): just row, which fuction check
    Returns:
        float: income
    '''
    if pd.isna(row['total_income']):
        return df_mediana_income.loc[row['income_type']]
    return row['total_income']


In [91]:
columns_name = ['income_type', 'total_income']  # Проверка работоспособности функции
values = [['безработный', 32432.324],['госслужащий', ]]
test_func_df = pd.DataFrame(data=values, columns=columns_name)
test_func_df['total_income'] = test_func_df.apply(change_income, axis=1)
test_func_df 

,income_type,total_income
0,безработный,32432.324000
1,госслужащий,150447.935283


In [92]:
df['total_income'] = df.apply(change_income, axis=1)
df.isna().sum() # Проверяем пропуски в зарплатах

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [93]:
def preproccesing_age(days_employed):
    '''function to try preprocessing empoyed

    Args:
        age (float): days_employed

    Returns:
        float: correct age in years
    '''
    return abs(days_employed) / 365.25 


# Будем надеятся что числа совпадают, знаки рандомны. попробую в "стаже" просто попробую привести к читабельному виду и к годам
# print(preproccesing_age(-1245.123)) # Проверка на жизнеспособность функции
df['days_employed'] = df['days_employed'].apply(preproccesing_age)

In [94]:
df['days_employed'].describe() 

count    19351.000000
mean       183.202543
std        380.645806
min          0.066096
25%          2.538013
50%          6.007449
75%         15.161896
max       1099.946339
Name: days_employed, dtype: float64

Много не корректных значений... Благо в дальнейшем исспледование столбец не понадобится.

In [95]:
df['days_employed'].head(3)

0    23.101090
1    11.019312
2    15.396092
Name: days_employed, dtype: float64

In [96]:
df['days_employed'] = df['days_employed'].fillna(0)  
df.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**  
Преположительная причина возникновения пропусков: человеческий фактор
Пропуски в зарплатах заменены на значение медианы для соответствующего типа занятости  
Для определения причины пропусков в данных по стажу, надо сначало понять, а чему они вообще соотносятся.  Данные относительно мусорные, было принято волевое решение пропуски заменить на нули, а на остальное забить. в дальнейшем исследовании не понадобятся. так вообще прослеживается что у пенсионеров в данный момент 1000, у молодежи показатели заметно меньше, тут надо гадать что бы это могло значить... Но при этом есть очень крутой разрыв...


### Замена типа данных

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [98]:
df['days_employed'] = df['days_employed'].astype('int')

**Вывод**

Смысла в замене не видел... нужные данные в правильных типах

### Обработка дубликатов

In [99]:
for i in ('education', 'family_status', 'income_type', 'purpose'):
    df[i] = df[i].str.lower()

df[df.duplicated()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,покупка жилья для семьи
3290,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
4182,1,0,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594.396847,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу


In [101]:
df = df.drop_duplicates()


In [102]:
df.shape

(21454, 12)

**Вывод**

Удалил полные дубли. считаю что шансов не много, что очень много параметров совпадут, а самое главное зарплата до тысячной. спишем природу этих дубликатов на технические неполадки.
В ручном поиске на данном этапе не вижу смысла искать дубликаты: 21454 значений и шансов что просто люди одного года выпуска и соц. статуса берут кредит на одно и тоже: высокий.

### Лемматизация

Тут два столбца требующих обработки текста: "Образование" и "Цель кредита". в первом случае должно хватить приведение к нижнему регистру

In [103]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [105]:
df['purpose'].unique()  # смотрим и категоризируем данные... 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Делим цели кредита на следующие категории:  
- Жилье
- Коммерческая недвижимость
- авто 
- образование
- авто 
- иное(вроде нет, но на всякий)  
Тут возникает проблема как поделить неопределенные значение "операции со своей недвижомостью", когда она не определена... Отнесем к коммерческой

In [106]:
# первоначально была попытка сделать с помощью pymystem3... в итоге оказалось что библиотека не оптимизирована под windows
# сделал через то что нашел в google...
import re
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()

In [108]:
def lemmating(row):
    '''function for lemmatize and categore string...

    Args:
        row (str): just string

    Returns:
        str: category of purpose
    '''
    lemmatize_word = morph.parse(row)[0].normal_form
    if 'недвижим' in lemmatize_word:
        if 'жил' in lemmatize_word:
            return 'Жилая недвижимость'
        else:
            return 'Коммерческая недвижимость'
    if 'жил' in lemmatize_word:
        return 'Жилая недвижимость'
    if 'авто' in lemmatize_word:
        return 'Автомобиль'
    if 'свад' in lemmatize_word:
        return 'Свадьба'
    if 'образован' in lemmatize_word or 'обучен' in lemmatize_word or 'учеб' in lemmatize_word:
        return 'Образование'
    return 'Иное'


# Тест работоспособности фукции:
# lemmating('покупка жилая')
df['purpose_category'] = df['purpose'].apply(lemmating)


In [109]:
df['purpose_category'].value_counts()

Жилая недвижимость           5690
Коммерческая недвижимость    5121
Автомобиль                   4306
Образование                  4013
Свадьба                      2324
Name: purpose_category, dtype: int64

**Вывод**

Категорийные данные приведены в порядок. леммитизированы и разбиты на категории, приведены к нижнему регистру.

### Категоризация данных

Имеет смысл объединить возраста по категориям, а то что-то большой разброс. Т.к. цель исследования: взаимосвязь с семейным положением, то категории сделать не очень крупными, а взять общепринятую классификацию по возрастам молодые – 9-17 лет; ранняя зрелость – 18-24 зрелость – 25-44 года средний пожилой возраст – 45-64 года; ранняя старость – 65-74 года и поздняя старость – 75 лет и старше. Но предварительно надо разобраться с нулями

In [111]:
df_age_zero = df[df['dob_years'] == 0] #смотрим сколько нулей и у кого... может не случайно
df_age_zero

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
99,0,948,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль,Автомобиль
149,0,7,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем,Жилая недвижимость
270,3,5,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью,Жилая недвижимость
578,0,1089,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости,Коммерческая недвижимость
1040,0,3,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль,Автомобиль
1149,0,2,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости,Коммерческая недвижимость
1175,0,1015,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования,Образование
1386,0,13,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем,Автомобиль
1890,0,0,0,высшее,0,не женат / не замужем,4,F,сотрудник,0,142594.396847,жилье,Жилая недвижимость
1898,0,1013,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля,Автомобиль


полистав df в jupiter variables: большинство пропусков у женщин... не удивлен. Будем заполнять по среднему значению для данного типа занятости... 

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  int32  
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  purpose_category  21454 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(6)
memory usage: 2.2+ MB


In [116]:
df_mean_age = df.groupby('income_type')['dob_years'].mean() 
df_mean_age = df_mean_age.astype('int')
def check_zero_age(age):
    '''function to check age, if zero, return mean age for this income type

    Args:
        age (int): age

    Returns:
        int: correct age
    '''     
    if age['dob_years'] != 0:
        return age['dob_years']
    return df_mean_age.loc[age['income_type']]

# columns_name = ['income_type', 'dob_years'] Проверка работоспособности функции
# values = [['безработный', 15],['госслужащий', 0]]
# test_func_df = pd.DataFrame(data=values, columns=columns_name)
# test_func_df['dob_years'] = test_func_df.apply(check_zero_age, axis=1)
# test_func_df 


In [119]:
df['dob_years'] = df.apply(check_zero_age, axis=1)
len(df[df['dob_years'] == 0])  # проверил что удалилось


0

In [120]:
def age_category(age):  #функция разбивки на категории
    '''fuction to differencing age people to category, if age < 18 - return None for drop letter

    Args:
        age (int): age of man
    return:
        age_category(str): category of age
    '''
    if age <= 17:
        return 'молодость'
    elif age <= 24:
        return 'ранняя зрелость'
    elif age <= 44:
        return 'зрелость'
    elif age <= 64:
        return 'средний пожилой возраст'
    elif age <= 74:
        return 'ранняя старость'
    else: 
        return 'поздняя старость'
    и
# age_category(75) проверка

In [121]:
df['age_category'] = df['dob_years'].apply(age_category)
df['age_category'].value_counts()

зрелость                   10913
средний пожилой возраст     8771
ранняя старость              894
ранняя зрелость              875
поздняя старость               1
Name: age_category, dtype: int64

In [122]:
df = df[df['age_category'] != 'поздняя старость'] # можно наверно было заменить на ранюю старость... но 1значение... не имеет значения

In [123]:
# проверка что данные категоризированы и нет расхождений. если бы были: надо было бы еще покапаться
display(df.pivot_table(index=['education'], aggfunc='mean', values='education_id'))
display(df.pivot_table(index=['family_status'], aggfunc='mean', values='family_status_id'))

,education_id
education,
высшее,0
начальное,3
неоконченное высшее,2
среднее,1
ученая степень,4


,family_status_id
family_status,
в разводе,3
вдовец / вдова,2
гражданский брак,1
женат / замужем,0
не женат / не замужем,4


In [124]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [125]:
df_strange_childrens = pd.concat([df[df['children'] == -1], df[df['children'] == 20]]) # создал df со стремными значениями детей
len(df_strange_childrens) # смотрю длину и папаллельно в viewer'е                            

123

In [126]:
df = df[df['children'] != 20] # удаляем, т.к. данных не много и распихивать кажется не корректным
df = df[df['children'] != -1] # тоже самое

In [127]:
def children_category(children):
    '''check children. if person have children return 1, else 0

    Args:
        children (int): sum childrens of person

    Returns:
        int: one or zero
    '''
    if children == 0:
        return 'Нет детей'
    else:
        return 'С детьми'

In [128]:
df['children_availability'] = df['children'].apply(children_category)

In [129]:
def category_purpose(purpose):
    if purpose < 10000:
        return 'A1: Крайняя нищета'
    if purpose < 15000:
        return 'A2: Беднейшие'
    if purpose < 18000:
        return 'A3: Бедные'
    elif purpose < 36000:
        return 'A4: Выше бедности'
    elif purpose < 100000:
        return 'A5: Нижний средний класс'
    elif purpose < 150000:
        return 'A6: Предсредний класс'
    elif purpose < 250000:
        return 'A7: Средний класс'
    elif purpose < 500000:
        return 'A8: Верхний средний класс'
    elif purpose < 1000000:
        return 'A9: Состоятельные'
    else:
        return 'A10: Богатые'

In [130]:
df['category_income'] = df['total_income'].apply(category_purpose)
df['category_income'].value_counts()

A6: Предсредний класс        7110
A7: Средний класс            6979
A5: Нижний средний класс     4369
A8: Верхний средний класс    2575
A9: Состоятельные             197
A4: Выше бедности              75
A10: Богатые                   25
Name: category_income, dtype: int64

т.к довольно мало богатых и выборка будет не показатальна. то объединим богатых,состоятельных с "Верхний средний класс", а также выше бедности и предсредний класс

In [131]:
df['category_income'] = df['category_income'].replace(['A9: Состоятельные', 'A10: Богатые'], 'A8: Верхний средний класс')
df['category_income'] = df['category_income'].replace('A4: Выше бедности', 'A5: Нижний средний класс')
df['category_income'].value_counts()

A6: Предсредний класс        7110
A7: Средний класс            6979
A5: Нижний средний класс     4444
A8: Верхний средний класс    2797
Name: category_income, dtype: int64

**Вывод**

- Данные по возрасту категоризированы с нулями разобрались, никаких сверхъестевственных выводов. разву что Львиная доля клиентов банка 24-64... при этом 24-44 чуть больше чем но 44-64... но пока принципиально это не на что не влияет.    
- Также сделана дополнительно категоризация "есть дети"/"нет детей". 
- Проиведена категоризация данных по зарплате согласно статье [Яндекс Дзен](https://zen.yandex.ru/media/dogecrypto/kak-poschitat-k-kakomu-klassu-vy-otnosites-po-urovniu-dohoda-5b150b5aea0fe700a848e1b1)

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [132]:
display(df.groupby('children')['debt'].agg(['mean', 'count']))
display(df.groupby('children_availability')['debt'].mean()*100)
len(df[df['children'] == 5])

,mean,count
children,,
0,0.075444,14090
1,0.092346,4808
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9


children_availability
Нет детей    7.544358
С детьми     9.240331
Name: debt, dtype: float64

9

**Вывод**

Наблюдается зависимость надежности клиентов от наличия детей. Однако кол-во детей на надежность не влияет.
Как видим есть зависимость от кол-ва детей и возвратами кредитов... Люди с детьми более чем в 1.6% случаев чаще имеют проблемы с оплатой по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [133]:
data_pivot_family_status = df.pivot_table(index=['family_status'], columns='children_availability', values='debt', aggfunc='mean', margins=True)
data_pivot_family_status

children_availability,Нет детей,С детьми,All
family_status,,,
в разводе,0.070153,0.071605,0.070648
вдовец / вдова,0.062648,0.096154,0.066316
гражданский брак,0.083883,0.111111,0.093130
женат / замужем,0.069095,0.085750,0.075606
не женат / не замужем,0.092838,0.117978,0.097639
All,0.075444,0.092403,0.081200


**Вывод**

Да. завимость есть... те кто в браке не побывал: кредиты менее охотно возвращают

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [134]:
data_pivot_income = df.pivot_table(index=['category_income'], columns='children_availability', values='debt', aggfunc='mean', margins=True)
data_pivot_income

children_availability,Нет детей,С детьми,All
category_income,,,
A5: Нижний средний класс,0.072435,0.095477,0.079658
A6: Предсредний класс,0.080367,0.100041,0.087060
A7: Средний класс,0.077245,0.087885,0.080957
A8: Верхний средний класс,0.063193,0.080564,0.069360
All,0.075444,0.092403,0.081200


**Вывод**

Зависимость очень плохо прослеживается. разве что люди с достатком свыше 250т.р. чуть реже имеют проблемы с погашением кредита. Возможно если бы это была основная цель исследования,необходимо было бы сделать более мелкую градацию и уточнить у банка как они производят дифференцируют клиентов по доходу.

- Как разные цели кредита влияют на его возврат в срок?

In [135]:
data_pivot_purpose = df.pivot_table(index=['purpose_category'], columns='children_availability', values='debt', aggfunc='mean', margins=True)
data_pivot_purpose

children_availability,Нет детей,С детьми,All
purpose_category,,,
Автомобиль,0.085413,0.109484,0.093480
Жилая недвижимость,0.067724,0.074819,0.070154
Коммерческая недвижимость,0.066806,0.091432,0.075216
Образование,0.086710,0.104012,0.092551
Свадьба,0.075163,0.086845,0.079118
All,0.075444,0.092403,0.081200


**Вывод**

основные задолжники: те кто тратиться на обучение и автомобили, а те то с детьми берут на авто: этим вообще не стоит доверять

## Шаг 4. Общий вывод

In [136]:
data_pivot_purpose_and_family = df.pivot_table(index=['purpose_category', 'family_status'], columns='children_availability', values='debt', aggfunc=('mean','count'))
data_pivot_purpose_and_family

count               mean  \
children_availability                           Нет детей С детьми Нет детей   
purpose_category          family_status                                        
Автомобиль                в разводе                   195       84  0.066667   
                          вдовец / вдова              201       17  0.094527   
                          гражданский брак            282      148  0.099291   
                          женат / замужем            1649     1068  0.072165   
                          не женат / не замужем       518      117  0.123552   
Жилая недвижимость        в разводе                   237      106  0.067511   
                          вдовец / вдова              243       38  0.041152   
                          гражданский брак            329      180  0.082067   
                          женат / замужем            2217     1458  0.068110   
                          не женат / не замужем       695      156  0.069065   
Коммерческая недвижимость в разводе                   194      135  0.067010   
                          вдовец / вдова              227       27  0.052863   
                          гражданский брак            320      159  0.075000   
                          женат / замужем            2020     1275  0.063366   
                          не женат / не замужем       592      143  0.079392   
Образование               в разводе                   158       80  0.082278   
                          вдовец / вдова              175       22  0.068571   
                          гражданский брак            269      134  0.130112   
                          женат / замужем            1582      992  0.074589   
                          не женат / не замужем       457      118  0.111597   
Свадьба                   гражданский брак           1530      783  0.075163   

                                                           
children_availability                            С детьми  
purpose_category          family_status                    
Автомобиль                в разводе              0.095238  
                          вдовец / вдова         0.058824  
                          гражданский брак       0.155405  
                          женат / замужем        0.100187  
                          не женат / не замужем  0.153846  
Жилая недвижимость        в разводе              0.056604  
                          вдовец / вдова         0.105263  
                          гражданский брак       0.122222  
                          женат / замужем        0.067901  
                          не женат / не замужем  0.089744  
Коммерческая недвижимость в разводе              0.081481  
                          вдовец / вдова         0.074074  
                          гражданский брак       0.113208  
                          женат / замужем        0.084706  
                          не женат / не замужем  0.139860  
Образование               в разводе              0.050000  
                          вдовец / вдова         0.136364  
                          гражданский брак       0.186567  
                          женат / замужем        0.097782  
                          не женат / не замужем  0.093220  
Свадьба                   гражданский брак       0.086845

В результате исспледований обе гипотезы подтвердились:  
- наличие детей отражается на потенциальной надежности клиентов банка. В среднем на 1.5 % люди с детьми имеют проблемы с оплатой кредита
- люди побывавшие в браке более надежные клиенты. 
Также наблюдается отражения надежности клиентов в в завимости от целей кредита. так если цель кредита обучение или какая то сделка с автомобилем: в среднем также на 1.5% чаще возникают проблемы с погашением кредита  
При сведение целей кредита, семейного положения и наличия детей: гипотезы также подтвердились, так в среднем человек с ребенком в гражданском браке берущий кредит 
на образование имеет проблемы в среднем в 18.5% случаев... и наоборот человек в официальном браке берущий кредит на сделку с недвижимостью имеет проблемы только в 6% случаев. Однако стоит отметить что данных для такой выборке маловато и для использования в дальнейшем желательно перепроверить на большем объеме...